In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
alldata = pd.concat([train, test],axis=0)

In [4]:
alldata.shape

(1309, 12)

In [5]:
# import pandas_profiling as pdp

# pdp.ProfileReport(alldata)

In [6]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
alldata['Family']=alldata['SibSp']+alldata['Parch']

In [8]:
alldata=alldata.drop(['Ticket','Cabin','SibSp','Parch','PassengerId','Name'],axis=1)

In [9]:
ID = test['PassengerId']

In [10]:
alldata.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Embarked', 'Family'], dtype='object')

In [11]:
alldata['Age'] = alldata['Age'].fillna(np.nanmean(alldata['Age']))
alldata.isna().sum()

Survived    418
Pclass        0
Sex           0
Age           0
Fare          1
Embarked      2
Family        0
dtype: int64

In [12]:
alldata = pd.get_dummies(alldata)

In [13]:
train = alldata.iloc[:train.shape[0],:]
test = alldata.iloc[train.shape[0]:,:]

In [14]:
target_col = 'Survived'
feature_cols = [col for col in alldata.columns if col not in target_col]

In [15]:
X_train = train[feature_cols]
y_train = train[target_col]
X_test = test[feature_cols]

In [16]:
from sklearn.model_selection import train_test_split
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_train,y_train, test_size=0.3, random_state=0)

In [17]:
feature_cols

['Pclass',
 'Age',
 'Fare',
 'Family',
 'Sex_female',
 'Sex_male',
 'Embarked_C',
 'Embarked_Q',
 'Embarked_S']

In [18]:
from lightgbm import LGBMClassifier
LGBC=LGBMClassifier()

LGBC.fit(X_train1,y_train1)
print(LGBC.score(X_test1,y_test1))

0.8134328358208955


In [19]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import cross_val_score

XGBC=XGBClassifier()
LGBC=LGBMClassifier()
CATC=CatBoostClassifier()

clf_labels = ['XGBC', 'LGBC', 'CATC']

print('10 Fold cross validation\n')
for clf, label in zip([XGBC, LGBC, CATC],clf_labels):
    scores=cross_val_score(estimator=clf, X=X_train, y=y_train,
                           cv=10, scoring='roc_auc')#, scoring='mean_squared_error'
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]"
          % (scores.mean(), scores.std(), label))


10 Fold cross validation

ROC AUC: 0.85 (+/- 0.06) [XGBC]
ROC AUC: 0.86 (+/- 0.07) [LGBC]
Learning rate set to 0.009371
0:	learn: 0.6863820	total: 57ms	remaining: 56.9s
1:	learn: 0.6797418	total: 58.1ms	remaining: 29s
2:	learn: 0.6749976	total: 58.8ms	remaining: 19.5s
3:	learn: 0.6700202	total: 59.5ms	remaining: 14.8s
4:	learn: 0.6641108	total: 60.7ms	remaining: 12.1s
5:	learn: 0.6581384	total: 61.9ms	remaining: 10.3s
6:	learn: 0.6528547	total: 62.7ms	remaining: 8.9s
7:	learn: 0.6464622	total: 64ms	remaining: 7.93s
8:	learn: 0.6407201	total: 65.2ms	remaining: 7.17s
9:	learn: 0.6354372	total: 66.3ms	remaining: 6.57s
10:	learn: 0.6300502	total: 67.5ms	remaining: 6.07s
11:	learn: 0.6249850	total: 68.7ms	remaining: 5.66s
12:	learn: 0.6206670	total: 69.5ms	remaining: 5.28s
13:	learn: 0.6154107	total: 70.8ms	remaining: 4.98s
14:	learn: 0.6109606	total: 72ms	remaining: 4.73s
15:	learn: 0.6055327	total: 73.1ms	remaining: 4.49s
16:	learn: 0.6009297	total: 74.1ms	remaining: 4.29s
17:	learn: 0.59

259:	learn: 0.3550876	total: 312ms	remaining: 887ms
260:	learn: 0.3549955	total: 312ms	remaining: 884ms
261:	learn: 0.3547453	total: 313ms	remaining: 883ms
262:	learn: 0.3545821	total: 314ms	remaining: 881ms
263:	learn: 0.3542680	total: 315ms	remaining: 880ms
264:	learn: 0.3540419	total: 317ms	remaining: 878ms
265:	learn: 0.3538390	total: 318ms	remaining: 876ms
266:	learn: 0.3535219	total: 319ms	remaining: 876ms
267:	learn: 0.3531555	total: 320ms	remaining: 874ms
268:	learn: 0.3529354	total: 321ms	remaining: 873ms
269:	learn: 0.3525092	total: 322ms	remaining: 871ms
270:	learn: 0.3521885	total: 323ms	remaining: 869ms
271:	learn: 0.3519263	total: 324ms	remaining: 868ms
272:	learn: 0.3517153	total: 325ms	remaining: 866ms
273:	learn: 0.3514743	total: 326ms	remaining: 865ms
274:	learn: 0.3512451	total: 327ms	remaining: 863ms
275:	learn: 0.3510247	total: 328ms	remaining: 861ms
276:	learn: 0.3508424	total: 330ms	remaining: 860ms
277:	learn: 0.3506716	total: 331ms	remaining: 859ms
278:	learn: 

424:	learn: 0.3265276	total: 479ms	remaining: 648ms
425:	learn: 0.3262500	total: 480ms	remaining: 647ms
426:	learn: 0.3261398	total: 481ms	remaining: 646ms
427:	learn: 0.3260657	total: 482ms	remaining: 645ms
428:	learn: 0.3259898	total: 483ms	remaining: 643ms
429:	learn: 0.3258435	total: 485ms	remaining: 642ms
430:	learn: 0.3257435	total: 486ms	remaining: 641ms
431:	learn: 0.3256683	total: 487ms	remaining: 640ms
432:	learn: 0.3255440	total: 488ms	remaining: 639ms
433:	learn: 0.3252689	total: 489ms	remaining: 637ms
434:	learn: 0.3252504	total: 489ms	remaining: 636ms
435:	learn: 0.3251311	total: 491ms	remaining: 635ms
436:	learn: 0.3249090	total: 492ms	remaining: 633ms
437:	learn: 0.3249083	total: 492ms	remaining: 631ms
438:	learn: 0.3247960	total: 493ms	remaining: 630ms
439:	learn: 0.3245696	total: 494ms	remaining: 629ms
440:	learn: 0.3244743	total: 495ms	remaining: 628ms
441:	learn: 0.3244313	total: 496ms	remaining: 627ms
442:	learn: 0.3243998	total: 497ms	remaining: 625ms
443:	learn: 

586:	learn: 0.3077404	total: 647ms	remaining: 456ms
587:	learn: 0.3075991	total: 649ms	remaining: 454ms
588:	learn: 0.3075386	total: 650ms	remaining: 453ms
589:	learn: 0.3074320	total: 651ms	remaining: 452ms
590:	learn: 0.3072903	total: 652ms	remaining: 451ms
591:	learn: 0.3071060	total: 653ms	remaining: 450ms
592:	learn: 0.3070237	total: 654ms	remaining: 449ms
593:	learn: 0.3068981	total: 655ms	remaining: 448ms
594:	learn: 0.3067049	total: 656ms	remaining: 447ms
595:	learn: 0.3065516	total: 657ms	remaining: 446ms
596:	learn: 0.3062068	total: 658ms	remaining: 444ms
597:	learn: 0.3060974	total: 659ms	remaining: 443ms
598:	learn: 0.3058862	total: 661ms	remaining: 442ms
599:	learn: 0.3056905	total: 662ms	remaining: 441ms
600:	learn: 0.3055621	total: 663ms	remaining: 440ms
601:	learn: 0.3055128	total: 664ms	remaining: 439ms
602:	learn: 0.3052963	total: 665ms	remaining: 438ms
603:	learn: 0.3051758	total: 666ms	remaining: 437ms
604:	learn: 0.3050419	total: 667ms	remaining: 435ms
605:	learn: 

746:	learn: 0.2902142	total: 816ms	remaining: 276ms
747:	learn: 0.2899884	total: 817ms	remaining: 275ms
748:	learn: 0.2899124	total: 818ms	remaining: 274ms
749:	learn: 0.2898575	total: 819ms	remaining: 273ms
750:	learn: 0.2896267	total: 820ms	remaining: 272ms
751:	learn: 0.2895673	total: 822ms	remaining: 271ms
752:	learn: 0.2895101	total: 823ms	remaining: 270ms
753:	learn: 0.2893845	total: 824ms	remaining: 269ms
754:	learn: 0.2891845	total: 825ms	remaining: 268ms
755:	learn: 0.2890716	total: 826ms	remaining: 267ms
756:	learn: 0.2889694	total: 827ms	remaining: 266ms
757:	learn: 0.2889153	total: 828ms	remaining: 264ms
758:	learn: 0.2888633	total: 829ms	remaining: 263ms
759:	learn: 0.2887752	total: 830ms	remaining: 262ms
760:	learn: 0.2886340	total: 831ms	remaining: 261ms
761:	learn: 0.2885605	total: 832ms	remaining: 260ms
762:	learn: 0.2885039	total: 834ms	remaining: 259ms
763:	learn: 0.2883423	total: 835ms	remaining: 258ms
764:	learn: 0.2882080	total: 836ms	remaining: 257ms
765:	learn: 

...

330:	learn: 0.3544763	total: 335ms	remaining: 677ms
331:	learn: 0.3543243	total: 336ms	remaining: 676ms
332:	learn: 0.3541944	total: 337ms	remaining: 674ms
333:	learn: 0.3540755	total: 338ms	remaining: 673ms
334:	learn: 0.3540417	total: 338ms	remaining: 672ms
335:	learn: 0.3538786	total: 339ms	remaining: 671ms
336:	learn: 0.3535028	total: 340ms	remaining: 670ms
337:	learn: 0.3533115	total: 341ms	remaining: 669ms
338:	learn: 0.3531579	total: 342ms	remaining: 668ms
339:	learn: 0.3530206	total: 343ms	remaining: 667ms
340:	learn: 0.3528465	total: 345ms	remaining: 666ms
341:	learn: 0.3527724	total: 346ms	remaining: 665ms
342:	learn: 0.3525570	total: 347ms	remaining: 664ms
343:	learn: 0.3524540	total: 348ms	remaining: 664ms
344:	learn: 0.3522389	total: 349ms	remaining: 663ms
345:	learn: 0.3519297	total: 350ms	remaining: 662ms
346:	learn: 0.3518412	total: 351ms	remaining: 661ms
347:	learn: 0.3516419	total: 352ms	remaining: 660ms
348:	learn: 0.3515635	total: 353ms	remaining: 659ms
349:	learn: 

495:	learn: 0.3331039	total: 503ms	remaining: 511ms
496:	learn: 0.3328881	total: 504ms	remaining: 510ms
497:	learn: 0.3326093	total: 505ms	remaining: 509ms
498:	learn: 0.3324590	total: 506ms	remaining: 508ms
499:	learn: 0.3324365	total: 506ms	remaining: 506ms
500:	learn: 0.3323857	total: 507ms	remaining: 505ms
501:	learn: 0.3322895	total: 508ms	remaining: 504ms
502:	learn: 0.3320450	total: 510ms	remaining: 503ms
503:	learn: 0.3319472	total: 511ms	remaining: 502ms
504:	learn: 0.3318950	total: 512ms	remaining: 502ms
505:	learn: 0.3317821	total: 513ms	remaining: 501ms
506:	learn: 0.3316583	total: 514ms	remaining: 500ms
507:	learn: 0.3313723	total: 515ms	remaining: 499ms
508:	learn: 0.3313571	total: 515ms	remaining: 497ms
509:	learn: 0.3313079	total: 516ms	remaining: 496ms
510:	learn: 0.3312171	total: 518ms	remaining: 495ms
511:	learn: 0.3310181	total: 519ms	remaining: 494ms
512:	learn: 0.3308936	total: 520ms	remaining: 493ms
513:	learn: 0.3307715	total: 521ms	remaining: 492ms
514:	learn: 

655:	learn: 0.3146427	total: 670ms	remaining: 352ms
656:	learn: 0.3144938	total: 671ms	remaining: 351ms
657:	learn: 0.3144273	total: 672ms	remaining: 350ms
658:	learn: 0.3142765	total: 674ms	remaining: 349ms
659:	learn: 0.3141504	total: 675ms	remaining: 348ms
660:	learn: 0.3140848	total: 676ms	remaining: 347ms
661:	learn: 0.3140329	total: 677ms	remaining: 345ms
662:	learn: 0.3138074	total: 678ms	remaining: 344ms
663:	learn: 0.3136410	total: 679ms	remaining: 344ms
664:	learn: 0.3136117	total: 680ms	remaining: 342ms
665:	learn: 0.3135027	total: 681ms	remaining: 341ms
666:	learn: 0.3133258	total: 682ms	remaining: 340ms
667:	learn: 0.3132181	total: 683ms	remaining: 339ms
668:	learn: 0.3131698	total: 684ms	remaining: 338ms
669:	learn: 0.3130664	total: 685ms	remaining: 337ms
670:	learn: 0.3129415	total: 686ms	remaining: 336ms
671:	learn: 0.3128707	total: 687ms	remaining: 335ms
672:	learn: 0.3128388	total: 688ms	remaining: 334ms
673:	learn: 0.3126790	total: 689ms	remaining: 333ms
674:	learn: 

818:	learn: 0.2976741	total: 838ms	remaining: 185ms
819:	learn: 0.2975454	total: 840ms	remaining: 184ms
820:	learn: 0.2973191	total: 841ms	remaining: 183ms
821:	learn: 0.2971637	total: 842ms	remaining: 182ms
822:	learn: 0.2969952	total: 843ms	remaining: 181ms
823:	learn: 0.2969141	total: 844ms	remaining: 180ms
824:	learn: 0.2967879	total: 845ms	remaining: 179ms
825:	learn: 0.2966950	total: 846ms	remaining: 178ms
826:	learn: 0.2966392	total: 847ms	remaining: 177ms
827:	learn: 0.2965312	total: 848ms	remaining: 176ms
828:	learn: 0.2964550	total: 849ms	remaining: 175ms
829:	learn: 0.2961201	total: 850ms	remaining: 174ms
830:	learn: 0.2960818	total: 851ms	remaining: 173ms
831:	learn: 0.2959638	total: 852ms	remaining: 172ms
832:	learn: 0.2958508	total: 853ms	remaining: 171ms
833:	learn: 0.2957435	total: 854ms	remaining: 170ms
834:	learn: 0.2956673	total: 855ms	remaining: 169ms
835:	learn: 0.2955704	total: 857ms	remaining: 168ms
836:	learn: 0.2954752	total: 858ms	remaining: 167ms
837:	learn: 

982:	learn: 0.2815700	total: 1.01s	remaining: 17.4ms
983:	learn: 0.2814178	total: 1.01s	remaining: 16.4ms
984:	learn: 0.2813251	total: 1.01s	remaining: 15.4ms
985:	learn: 0.2813073	total: 1.01s	remaining: 14.4ms
986:	learn: 0.2812312	total: 1.01s	remaining: 13.3ms
987:	learn: 0.2811551	total: 1.01s	remaining: 12.3ms
988:	learn: 0.2811132	total: 1.01s	remaining: 11.3ms
989:	learn: 0.2810204	total: 1.02s	remaining: 10.3ms
990:	learn: 0.2809207	total: 1.02s	remaining: 9.24ms
991:	learn: 0.2807606	total: 1.02s	remaining: 8.21ms
992:	learn: 0.2807156	total: 1.02s	remaining: 7.18ms
993:	learn: 0.2806529	total: 1.02s	remaining: 6.16ms
994:	learn: 0.2805768	total: 1.02s	remaining: 5.13ms
995:	learn: 0.2804318	total: 1.02s	remaining: 4.11ms
996:	learn: 0.2802560	total: 1.02s	remaining: 3.08ms
997:	learn: 0.2802410	total: 1.02s	remaining: 2.05ms
998:	learn: 0.2801568	total: 1.02s	remaining: 1.03ms
999:	learn: 0.2801135	total: 1.03s	remaining: 0us
ROC AUC: 0.87 (+/- 0.06) [CATC]


In [20]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator


class MajorityVoteClassifier(BaseEstimator, 
                             ClassifierMixin):
    """ A majority vote ensemble classifier

    Parameters
    ----------
    classifiers : array-like, shape = [n_classifiers]
      Different classifiers for the ensemble

    vote : str, {'classlabel', 'probability'} (default='classlabel')
      If 'classlabel' the prediction is based on the argmax of
        class labels. Else if 'probability', the argmax of
        the sum of probabilities is used to predict the class label
        (recommended for calibrated classifiers).

    weights : array-like, shape = [n_classifiers], optional (default=None)
      If a list of `int` or `float` values are provided, the classifiers
      are weighted by importance; Uses uniform weights if `weights=None`.

    """
    def __init__(self, classifiers, vote='classlabel', weights=None):

        self.classifiers = classifiers
        self.named_classifiers = {key: value for key, value
                                  in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):
        """ Fit classifiers.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Matrix of training examples.

        y : array-like, shape = [n_examples]
            Vector of target class labels.

        Returns
        -------
        self : object

        """
        if self.vote not in ('probability', 'classlabel'):
            raise ValueError("vote must be 'probability' or 'classlabel'"
                             "; got (vote=%r)"
                             % self.vote)

        if self.weights and len(self.weights) != len(self.classifiers):
            raise ValueError('Number of classifiers and weights must be equal'
                             '; got %d weights, %d classifiers'
                             % (len(self.weights), len(self.classifiers)))

        # Use LabelEncoder to ensure class labels start with 0, which
        # is important for np.argmax call in self.predict
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

    def predict(self, X):
        """ Predict class labels for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Matrix of training examples.

        Returns
        ----------
        maj_vote : array-like, shape = [n_examples]
            Predicted class labels.
            
        """
        if self.vote == 'probability':
            maj_vote = np.argmax(self.predict_proba(X), axis=1)
        else:  # 'classlabel' vote

            #  Collect results from clf.predict calls
            predictions = np.asarray([clf.predict(X)
                                      for clf in self.classifiers_]).T

            maj_vote = np.apply_along_axis(
                                      lambda x:
                                      np.argmax(np.bincount(x,
                                                weights=self.weights)),
                                      axis=1,
                                      arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote

    def predict_proba(self, X):
        """ Predict class probabilities for X.

        Parameters
        ----------
        X : {array-like, sparse matrix}, shape = [n_examples, n_features]
            Training vectors, where n_examples is the number of examples and
            n_features is the number of features.

        Returns
        ----------
        avg_proba : array-like, shape = [n_examples, n_classes]
            Weighted average probability for each class per example.

        """
        probas = np.asarray([clf.predict_proba(X)
                             for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba

    def get_params(self, deep=True):
        """ Get classifier parameter names for GridSearch"""
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name, step in self.named_classifiers.items():
                for key, value in step.get_params(deep=True).items():
                    out['%s__%s' % (name, key)] = value
            return out

In [21]:
mv_clf = MajorityVoteClassifier(classifiers=[XGBC, LGBC, CATC])

clf_labels = clf_labels + ['Majority voting']
all_clf = [XGBC, LGBC, CATC, mv_clf]

for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator=clf, X=X_train, y=y_train,
                             cv=10, scoring='roc_auc')
    print('ROC AUC: %0.2f (± %0.2f)[%s]' % (scores.mean(), scores.std(), label))

ROC AUC: 0.85 (± 0.06)[XGBC]
ROC AUC: 0.86 (± 0.07)[LGBC]
Learning rate set to 0.009371
0:	learn: 0.6863820	total: 1.48ms	remaining: 1.48s
1:	learn: 0.6797418	total: 2.69ms	remaining: 1.34s
2:	learn: 0.6749976	total: 3.39ms	remaining: 1.13s
3:	learn: 0.6700202	total: 4.07ms	remaining: 1.01s
4:	learn: 0.6641108	total: 5.23ms	remaining: 1.04s
5:	learn: 0.6581384	total: 6.38ms	remaining: 1.06s
6:	learn: 0.6528547	total: 7.2ms	remaining: 1.02s
7:	learn: 0.6464622	total: 8.41ms	remaining: 1.04s
8:	learn: 0.6407201	total: 9.61ms	remaining: 1.06s
9:	learn: 0.6354372	total: 10.8ms	remaining: 1.07s
10:	learn: 0.6300502	total: 12ms	remaining: 1.08s
11:	learn: 0.6249850	total: 13.2ms	remaining: 1.08s
12:	learn: 0.6206670	total: 14ms	remaining: 1.06s
13:	learn: 0.6154107	total: 15.1ms	remaining: 1.06s
14:	learn: 0.6109606	total: 16.3ms	remaining: 1.07s
15:	learn: 0.6055327	total: 17.5ms	remaining: 1.07s
16:	learn: 0.6009297	total: 18.7ms	remaining: 1.08s
17:	learn: 0.5960409	total: 19.8ms	remainin

297:	learn: 0.3467848	total: 309ms	remaining: 729ms
298:	learn: 0.3464958	total: 311ms	remaining: 728ms
299:	learn: 0.3464137	total: 312ms	remaining: 727ms
300:	learn: 0.3460872	total: 313ms	remaining: 726ms
301:	learn: 0.3457789	total: 314ms	remaining: 725ms
302:	learn: 0.3455829	total: 315ms	remaining: 724ms
303:	learn: 0.3454104	total: 316ms	remaining: 723ms
304:	learn: 0.3452740	total: 317ms	remaining: 722ms
305:	learn: 0.3451751	total: 318ms	remaining: 721ms
306:	learn: 0.3449349	total: 319ms	remaining: 720ms
307:	learn: 0.3446670	total: 320ms	remaining: 719ms
308:	learn: 0.3444364	total: 321ms	remaining: 718ms
309:	learn: 0.3442639	total: 322ms	remaining: 717ms
310:	learn: 0.3439958	total: 323ms	remaining: 716ms
311:	learn: 0.3437490	total: 324ms	remaining: 716ms
312:	learn: 0.3435324	total: 326ms	remaining: 715ms
313:	learn: 0.3433669	total: 327ms	remaining: 714ms
314:	learn: 0.3431662	total: 328ms	remaining: 713ms
315:	learn: 0.3430132	total: 329ms	remaining: 712ms
316:	learn: 

459:	learn: 0.3223486	total: 475ms	remaining: 557ms
460:	learn: 0.3221192	total: 476ms	remaining: 556ms
461:	learn: 0.3218179	total: 477ms	remaining: 555ms
462:	learn: 0.3216957	total: 478ms	remaining: 554ms
463:	learn: 0.3215542	total: 479ms	remaining: 553ms
464:	learn: 0.3214171	total: 480ms	remaining: 552ms
465:	learn: 0.3213546	total: 481ms	remaining: 551ms
466:	learn: 0.3212853	total: 482ms	remaining: 550ms
467:	learn: 0.3211568	total: 483ms	remaining: 549ms
468:	learn: 0.3210083	total: 484ms	remaining: 548ms
469:	learn: 0.3208097	total: 485ms	remaining: 547ms
470:	learn: 0.3207100	total: 486ms	remaining: 546ms
471:	learn: 0.3206327	total: 487ms	remaining: 545ms
472:	learn: 0.3205115	total: 488ms	remaining: 544ms
473:	learn: 0.3203367	total: 489ms	remaining: 543ms
474:	learn: 0.3201433	total: 491ms	remaining: 542ms
475:	learn: 0.3200511	total: 492ms	remaining: 541ms
476:	learn: 0.3199028	total: 493ms	remaining: 540ms
477:	learn: 0.3198382	total: 494ms	remaining: 539ms
478:	learn: 

626:	learn: 0.3025314	total: 642ms	remaining: 382ms
627:	learn: 0.3024515	total: 643ms	remaining: 381ms
628:	learn: 0.3023006	total: 644ms	remaining: 380ms
629:	learn: 0.3021750	total: 645ms	remaining: 379ms
630:	learn: 0.3018949	total: 647ms	remaining: 378ms
631:	learn: 0.3016734	total: 648ms	remaining: 377ms
632:	learn: 0.3016437	total: 648ms	remaining: 376ms
633:	learn: 0.3015694	total: 649ms	remaining: 375ms
634:	learn: 0.3014842	total: 650ms	remaining: 374ms
635:	learn: 0.3013980	total: 652ms	remaining: 373ms
636:	learn: 0.3012350	total: 653ms	remaining: 372ms
637:	learn: 0.3011243	total: 654ms	remaining: 371ms
638:	learn: 0.3010172	total: 655ms	remaining: 370ms
639:	learn: 0.3009200	total: 656ms	remaining: 369ms
640:	learn: 0.3009069	total: 657ms	remaining: 368ms
641:	learn: 0.3007391	total: 658ms	remaining: 367ms
642:	learn: 0.3005774	total: 659ms	remaining: 366ms
643:	learn: 0.3005313	total: 660ms	remaining: 365ms
644:	learn: 0.3004387	total: 661ms	remaining: 364ms
645:	learn: 

789:	learn: 0.2856301	total: 811ms	remaining: 215ms
790:	learn: 0.2855282	total: 812ms	remaining: 215ms
791:	learn: 0.2854874	total: 813ms	remaining: 213ms
792:	learn: 0.2853722	total: 814ms	remaining: 212ms
793:	learn: 0.2853663	total: 815ms	remaining: 211ms
794:	learn: 0.2853193	total: 816ms	remaining: 210ms
795:	learn: 0.2851819	total: 817ms	remaining: 209ms
796:	learn: 0.2851000	total: 818ms	remaining: 208ms
797:	learn: 0.2849791	total: 819ms	remaining: 207ms
798:	learn: 0.2849074	total: 820ms	remaining: 206ms
799:	learn: 0.2848620	total: 821ms	remaining: 205ms
800:	learn: 0.2847317	total: 822ms	remaining: 204ms
801:	learn: 0.2846829	total: 823ms	remaining: 203ms
802:	learn: 0.2846305	total: 824ms	remaining: 202ms
803:	learn: 0.2844655	total: 825ms	remaining: 201ms
804:	learn: 0.2843692	total: 827ms	remaining: 200ms
805:	learn: 0.2841657	total: 828ms	remaining: 199ms
806:	learn: 0.2840492	total: 829ms	remaining: 198ms
807:	learn: 0.2839958	total: 830ms	remaining: 197ms
808:	learn: 

...

179:	learn: 0.3882519	total: 187ms	remaining: 853ms
180:	learn: 0.3878658	total: 188ms	remaining: 852ms
181:	learn: 0.3875648	total: 189ms	remaining: 851ms
182:	learn: 0.3871803	total: 190ms	remaining: 850ms
183:	learn: 0.3869041	total: 191ms	remaining: 849ms
184:	learn: 0.3866303	total: 192ms	remaining: 847ms
185:	learn: 0.3862405	total: 193ms	remaining: 846ms
186:	learn: 0.3859102	total: 194ms	remaining: 845ms
187:	learn: 0.3856374	total: 195ms	remaining: 844ms
188:	learn: 0.3852366	total: 196ms	remaining: 843ms
189:	learn: 0.3849797	total: 197ms	remaining: 842ms
190:	learn: 0.3845652	total: 199ms	remaining: 841ms
191:	learn: 0.3842688	total: 200ms	remaining: 840ms
192:	learn: 0.3840776	total: 201ms	remaining: 839ms
193:	learn: 0.3838717	total: 202ms	remaining: 838ms
194:	learn: 0.3835128	total: 203ms	remaining: 837ms
195:	learn: 0.3832095	total: 204ms	remaining: 836ms
196:	learn: 0.3828303	total: 205ms	remaining: 835ms
197:	learn: 0.3825807	total: 206ms	remaining: 834ms
198:	learn: 

347:	learn: 0.3516419	total: 356ms	remaining: 667ms
348:	learn: 0.3515635	total: 357ms	remaining: 666ms
349:	learn: 0.3513420	total: 358ms	remaining: 665ms
350:	learn: 0.3511242	total: 359ms	remaining: 664ms
351:	learn: 0.3507881	total: 360ms	remaining: 663ms
352:	learn: 0.3506895	total: 361ms	remaining: 662ms
353:	learn: 0.3505008	total: 362ms	remaining: 661ms
354:	learn: 0.3503420	total: 364ms	remaining: 661ms
355:	learn: 0.3502789	total: 365ms	remaining: 660ms
356:	learn: 0.3500882	total: 366ms	remaining: 659ms
357:	learn: 0.3499150	total: 367ms	remaining: 658ms
358:	learn: 0.3497806	total: 368ms	remaining: 657ms
359:	learn: 0.3496979	total: 369ms	remaining: 656ms
360:	learn: 0.3494745	total: 370ms	remaining: 655ms
361:	learn: 0.3494270	total: 371ms	remaining: 653ms
362:	learn: 0.3492949	total: 372ms	remaining: 652ms
363:	learn: 0.3491537	total: 373ms	remaining: 651ms
364:	learn: 0.3490026	total: 374ms	remaining: 650ms
365:	learn: 0.3488905	total: 375ms	remaining: 649ms
366:	learn: 

514:	learn: 0.3307491	total: 523ms	remaining: 492ms
515:	learn: 0.3306746	total: 524ms	remaining: 491ms
516:	learn: 0.3305551	total: 525ms	remaining: 490ms
517:	learn: 0.3303875	total: 526ms	remaining: 489ms
518:	learn: 0.3302320	total: 527ms	remaining: 488ms
519:	learn: 0.3301619	total: 528ms	remaining: 488ms
520:	learn: 0.3300786	total: 529ms	remaining: 487ms
521:	learn: 0.3300167	total: 530ms	remaining: 486ms
522:	learn: 0.3299764	total: 531ms	remaining: 485ms
523:	learn: 0.3298959	total: 532ms	remaining: 484ms
524:	learn: 0.3297351	total: 533ms	remaining: 483ms
525:	learn: 0.3297226	total: 534ms	remaining: 481ms
526:	learn: 0.3295779	total: 535ms	remaining: 480ms
527:	learn: 0.3294631	total: 536ms	remaining: 479ms
528:	learn: 0.3292634	total: 537ms	remaining: 478ms
529:	learn: 0.3290703	total: 538ms	remaining: 477ms
530:	learn: 0.3289575	total: 539ms	remaining: 476ms
531:	learn: 0.3288938	total: 540ms	remaining: 475ms
532:	learn: 0.3287734	total: 541ms	remaining: 474ms
533:	learn: 

679:	learn: 0.3122940	total: 691ms	remaining: 325ms
680:	learn: 0.3122513	total: 692ms	remaining: 324ms
681:	learn: 0.3121261	total: 693ms	remaining: 323ms
682:	learn: 0.3120204	total: 694ms	remaining: 322ms
683:	learn: 0.3119690	total: 695ms	remaining: 321ms
684:	learn: 0.3118962	total: 696ms	remaining: 320ms
685:	learn: 0.3117296	total: 698ms	remaining: 319ms
686:	learn: 0.3116222	total: 699ms	remaining: 318ms
687:	learn: 0.3113374	total: 700ms	remaining: 317ms
688:	learn: 0.3110399	total: 701ms	remaining: 316ms
689:	learn: 0.3110044	total: 702ms	remaining: 315ms
690:	learn: 0.3109672	total: 703ms	remaining: 314ms
691:	learn: 0.3108338	total: 704ms	remaining: 313ms
692:	learn: 0.3107446	total: 705ms	remaining: 312ms
693:	learn: 0.3107010	total: 705ms	remaining: 311ms
694:	learn: 0.3106232	total: 706ms	remaining: 310ms
695:	learn: 0.3105143	total: 708ms	remaining: 309ms
696:	learn: 0.3104682	total: 709ms	remaining: 308ms
697:	learn: 0.3103687	total: 710ms	remaining: 307ms
698:	learn: 

844:	learn: 0.2946410	total: 860ms	remaining: 158ms
845:	learn: 0.2945844	total: 861ms	remaining: 157ms
846:	learn: 0.2944605	total: 862ms	remaining: 156ms
847:	learn: 0.2943446	total: 863ms	remaining: 155ms
848:	learn: 0.2942635	total: 864ms	remaining: 154ms
849:	learn: 0.2941617	total: 865ms	remaining: 153ms
850:	learn: 0.2940198	total: 866ms	remaining: 152ms
851:	learn: 0.2939328	total: 867ms	remaining: 151ms
852:	learn: 0.2938829	total: 868ms	remaining: 150ms
853:	learn: 0.2937008	total: 869ms	remaining: 149ms
854:	learn: 0.2936127	total: 870ms	remaining: 148ms
855:	learn: 0.2935320	total: 871ms	remaining: 147ms
856:	learn: 0.2934502	total: 872ms	remaining: 146ms
857:	learn: 0.2934466	total: 873ms	remaining: 144ms
858:	learn: 0.2933958	total: 874ms	remaining: 143ms
859:	learn: 0.2933164	total: 875ms	remaining: 142ms
860:	learn: 0.2931165	total: 876ms	remaining: 141ms
861:	learn: 0.2930633	total: 877ms	remaining: 140ms
862:	learn: 0.2929483	total: 878ms	remaining: 139ms
863:	learn: 

In [22]:
mv_clf.fit(X_train,y_train)

Learning rate set to 0.009807
0:	learn: 0.6868723	total: 2.21ms	remaining: 2.21s
1:	learn: 0.6803068	total: 3.44ms	remaining: 1.72s
2:	learn: 0.6753681	total: 4.2ms	remaining: 1.4s
3:	learn: 0.6702029	total: 5.01ms	remaining: 1.25s
4:	learn: 0.6641983	total: 6.28ms	remaining: 1.25s
5:	learn: 0.6587316	total: 7.55ms	remaining: 1.25s
6:	learn: 0.6533320	total: 8.39ms	remaining: 1.19s
7:	learn: 0.6472134	total: 9.61ms	remaining: 1.19s
8:	learn: 0.6411927	total: 10.8ms	remaining: 1.19s
9:	learn: 0.6357691	total: 12.1ms	remaining: 1.19s
10:	learn: 0.6311275	total: 13.4ms	remaining: 1.2s
11:	learn: 0.6258951	total: 14.6ms	remaining: 1.21s
12:	learn: 0.6214984	total: 15.6ms	remaining: 1.18s
13:	learn: 0.6158577	total: 16.8ms	remaining: 1.18s
14:	learn: 0.6111324	total: 18ms	remaining: 1.18s
15:	learn: 0.6057275	total: 19.3ms	remaining: 1.19s
16:	learn: 0.6009072	total: 20.6ms	remaining: 1.19s
17:	learn: 0.5960187	total: 21.9ms	remaining: 1.2s
18:	learn: 0.5919305	total: 23.2ms	remaining: 1.2s

304:	learn: 0.3542459	total: 336ms	remaining: 765ms
305:	learn: 0.3541813	total: 336ms	remaining: 763ms
306:	learn: 0.3539327	total: 338ms	remaining: 762ms
307:	learn: 0.3538158	total: 338ms	remaining: 760ms
308:	learn: 0.3536055	total: 340ms	remaining: 759ms
309:	learn: 0.3534238	total: 341ms	remaining: 758ms
310:	learn: 0.3531424	total: 342ms	remaining: 758ms
311:	learn: 0.3529498	total: 343ms	remaining: 757ms
312:	learn: 0.3527584	total: 344ms	remaining: 756ms
313:	learn: 0.3526566	total: 345ms	remaining: 754ms
314:	learn: 0.3524962	total: 346ms	remaining: 753ms
315:	learn: 0.3523534	total: 348ms	remaining: 752ms
316:	learn: 0.3522177	total: 349ms	remaining: 752ms
317:	learn: 0.3521010	total: 350ms	remaining: 751ms
318:	learn: 0.3518891	total: 351ms	remaining: 750ms
319:	learn: 0.3517214	total: 352ms	remaining: 748ms
320:	learn: 0.3515108	total: 353ms	remaining: 747ms
321:	learn: 0.3513618	total: 354ms	remaining: 746ms
322:	learn: 0.3511807	total: 355ms	remaining: 745ms
323:	learn: 

462:	learn: 0.3336139	total: 505ms	remaining: 586ms
463:	learn: 0.3335068	total: 506ms	remaining: 585ms
464:	learn: 0.3334803	total: 507ms	remaining: 583ms
465:	learn: 0.3334244	total: 508ms	remaining: 582ms
466:	learn: 0.3334119	total: 509ms	remaining: 581ms
467:	learn: 0.3334101	total: 509ms	remaining: 579ms
468:	learn: 0.3333995	total: 510ms	remaining: 577ms
469:	learn: 0.3332441	total: 511ms	remaining: 576ms
470:	learn: 0.3330482	total: 512ms	remaining: 575ms
471:	learn: 0.3328830	total: 513ms	remaining: 574ms
472:	learn: 0.3327477	total: 515ms	remaining: 574ms
473:	learn: 0.3326622	total: 516ms	remaining: 573ms
474:	learn: 0.3324593	total: 517ms	remaining: 571ms
475:	learn: 0.3323374	total: 518ms	remaining: 570ms
476:	learn: 0.3321855	total: 519ms	remaining: 569ms
477:	learn: 0.3320850	total: 520ms	remaining: 568ms
478:	learn: 0.3319308	total: 521ms	remaining: 567ms
479:	learn: 0.3318700	total: 523ms	remaining: 566ms
480:	learn: 0.3315841	total: 524ms	remaining: 565ms
481:	learn: 

621:	learn: 0.3165335	total: 675ms	remaining: 410ms
622:	learn: 0.3162426	total: 676ms	remaining: 409ms
623:	learn: 0.3161255	total: 677ms	remaining: 408ms
624:	learn: 0.3160241	total: 678ms	remaining: 407ms
625:	learn: 0.3159532	total: 679ms	remaining: 406ms
626:	learn: 0.3158138	total: 681ms	remaining: 405ms
627:	learn: 0.3155567	total: 682ms	remaining: 404ms
628:	learn: 0.3154704	total: 683ms	remaining: 403ms
629:	learn: 0.3153841	total: 684ms	remaining: 402ms
630:	learn: 0.3151810	total: 685ms	remaining: 401ms
631:	learn: 0.3150959	total: 686ms	remaining: 399ms
632:	learn: 0.3149654	total: 687ms	remaining: 398ms
633:	learn: 0.3148012	total: 688ms	remaining: 397ms
634:	learn: 0.3146928	total: 689ms	remaining: 396ms
635:	learn: 0.3146489	total: 690ms	remaining: 395ms
636:	learn: 0.3146482	total: 691ms	remaining: 394ms
637:	learn: 0.3145807	total: 692ms	remaining: 393ms
638:	learn: 0.3145029	total: 693ms	remaining: 391ms
639:	learn: 0.3144233	total: 694ms	remaining: 390ms
640:	learn: 

781:	learn: 0.3001404	total: 844ms	remaining: 235ms
782:	learn: 0.3000124	total: 845ms	remaining: 234ms
783:	learn: 0.2998708	total: 846ms	remaining: 233ms
784:	learn: 0.2997562	total: 847ms	remaining: 232ms
785:	learn: 0.2996874	total: 848ms	remaining: 231ms
786:	learn: 0.2995569	total: 849ms	remaining: 230ms
787:	learn: 0.2995096	total: 850ms	remaining: 229ms
788:	learn: 0.2994789	total: 851ms	remaining: 228ms
789:	learn: 0.2994368	total: 852ms	remaining: 227ms
790:	learn: 0.2991196	total: 853ms	remaining: 225ms
791:	learn: 0.2990631	total: 854ms	remaining: 224ms
792:	learn: 0.2989685	total: 856ms	remaining: 223ms
793:	learn: 0.2989075	total: 857ms	remaining: 222ms
794:	learn: 0.2988290	total: 858ms	remaining: 221ms
795:	learn: 0.2987602	total: 859ms	remaining: 220ms
796:	learn: 0.2986328	total: 860ms	remaining: 219ms
797:	learn: 0.2984931	total: 861ms	remaining: 218ms
798:	learn: 0.2983224	total: 862ms	remaining: 217ms
799:	learn: 0.2981920	total: 863ms	remaining: 216ms
800:	learn: 

941:	learn: 0.2846792	total: 1.01s	remaining: 62.4ms
942:	learn: 0.2845414	total: 1.01s	remaining: 61.3ms
943:	learn: 0.2844565	total: 1.01s	remaining: 60.2ms
944:	learn: 0.2843778	total: 1.02s	remaining: 59.2ms
945:	learn: 0.2843550	total: 1.02s	remaining: 58.1ms
946:	learn: 0.2843074	total: 1.02s	remaining: 57ms
947:	learn: 0.2842610	total: 1.02s	remaining: 55.9ms
948:	learn: 0.2841122	total: 1.02s	remaining: 54.9ms
949:	learn: 0.2840208	total: 1.02s	remaining: 53.8ms
950:	learn: 0.2839563	total: 1.02s	remaining: 52.7ms
951:	learn: 0.2839047	total: 1.02s	remaining: 51.6ms
952:	learn: 0.2838246	total: 1.02s	remaining: 50.6ms
953:	learn: 0.2837457	total: 1.03s	remaining: 49.5ms
954:	learn: 0.2836799	total: 1.03s	remaining: 48.4ms
955:	learn: 0.2835172	total: 1.03s	remaining: 47.3ms
956:	learn: 0.2832083	total: 1.03s	remaining: 46.3ms
957:	learn: 0.2831298	total: 1.03s	remaining: 45.2ms
958:	learn: 0.2831084	total: 1.03s	remaining: 44.1ms
959:	learn: 0.2830399	total: 1.03s	remaining: 43

MajorityVoteClassifier(classifiers=[XGBClassifier(base_score=None, booster=None,
                                                  colsample_bylevel=None,
                                                  colsample_bynode=None,
                                                  colsample_bytree=None,
                                                  gamma=None, gpu_id=None,
                                                  importance_type='gain',
                                                  interaction_constraints=None,
                                                  learning_rate=None,
                                                  max_delta_step=None,
                                                  max_depth=None,
                                                  min_child_weight=None,
                                                  missing=nan,
                                                  monotone_constraints=None,
                                                  

In [23]:
y_pred = mv_clf.predict(X_test)

In [24]:
submission = pd.DataFrame({'PassengerId':ID,'Survived':y_pred})

In [25]:
submission['Survived']=submission['Survived'].replace(0.0,'0').replace(1.0,'1')

In [26]:
submission.to_csv('submission.csv',index=False)

In [27]:
X_train1.isna().sum()

Pclass        0
Age           0
Fare          0
Family        0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [28]:
from sklearn.manifold import TSNE
X_reduced=TSNE(n_components=2).fit_transform(X_train1)

In [32]:
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import roc_curve, auc
from distutils.version import LooseVersion as Version
from scipy import __version__ as scipy_version


if scipy_version >= Version('1.4.1'):
    from numpy import interp
else:
    from scipy import interp


    

cv = list(StratifiedKFold(n_splits=3).split(X_train, y_train))
fig = plt.figure(figsize=(7, 5))

mean_tpr = 0.0
mean_fpr = np.linspace(0, 1, 100)
all_tpr = []

for i, (tra_idx, var_idx) in enumerate(cv):
    probas = pipe_lr.fit(X_train2[tra_idx],
                         y_train[tra_idx]).predict_proba(X_train2[var_idx])

    fpr, tpr, thresholds = roc_curve(y_train[test],
                                     probas[:, 1],
                                     pos_label=1)
    mean_tpr += interp(mean_fpr, fpr, tpr)
    mean_tpr[0] = 0.0
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr,
             tpr,
             label='ROC fold %d (area = %0.2f)'
                   % (i+1, roc_auc))

plt.plot([0, 1],
         [0, 1],
         linestyle='--',
         color=(0.6, 0.6, 0.6),
         label='Random guessing')

mean_tpr /= len(cv)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
plt.plot(mean_fpr, mean_tpr, 'k--',
         label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)
plt.plot([0, 0, 1],
         [0, 1, 1],
         linestyle=':',
         color='black',
         label='Perfect performance')

plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.legend(loc="lower right")

plt.tight_layout()
# plt.savefig('images/06_10.png', dpi=300)
plt.show()

NameError: name 'pipe_lr' is not defined

<Figure size 504x360 with 0 Axes>

In [ ]:
X_train1.isna().sum()